In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
train = pd.read_csv('./data/train_data.csv')
test = pd.read_csv('./data/test_data.csv')
sub = pd.read_csv('./data/sample_submission.csv')

In [3]:
print(train.sample(3))

       index                                            premise  \
13307  13307     이수역 사건을 두고 산이와 디스 공방을 벌인 제리케이가 대응하지 않겠다고 선언했다.   
13870  13870  유배지에서 윤필상이 죽었을 때 그의 시체를 열흘 동안이나 들판에 버려두고 방치하였는...   
12381  12381  유씨가 입국 거부와 관련해 법원에 소송을 낸 것은 이번이 처음으로 2002년 국가인...   

                             hypothesis          label  
13307           산이와 제리케이는 서로 디스를 주고받았다.     entailment  
13870                  윤필상은 유배지에서 병사했다.        neutral  
12381  유씨가 2002년 국가인권위원회에 낸 진정은 받아들여졌다.  contradiction  


In [4]:
train = train.dropna(how='any') # Null 값이 존재하는 행 제거
train = train.reset_index(drop=True) 
print(train.isnull().values.any()) # Null 값이 존재하는지 확인 

False


In [5]:
test = test.dropna(how='any') # Null 값이 존재하는 행 제거 
test = test.reset_index(drop=True)
print(test.isnull().values.any()) # Null 값이 존재하는지 확인 

False


In [6]:
print("premise 최대 길이:", train['premise'].map(len).max())
print("hypothesis 최대 길이:", train['hypothesis'].map(len).max())
print("premise 최대 길이:", test['premise'].map(len).max())
print("hypothesis 최대 길이:", test['hypothesis'].map(len).max())

premise 최대 길이: 90
hypothesis 최대 길이: 103
premise 최대 길이: 90
hypothesis 최대 길이: 75


In [7]:
max_seq_len = 100
valid = train[100:150]
train = train[200:250]
print(train)

     index                                            premise  \
200    200                   쏟아지는 빗속에서 길 잃은 새끼고양이 한 마리가 찾아든다.   
201    201  또한 저지대와 산사태, 축대 옹벽 등 붕괴 위험지역 예찰을 강화하고 위험 징후 시 ...   
202    202                           무엇보다 호스트의 따뜻한 배려에 감동받았다.   
203    203                          나쁜의미가 아니라, 각각의 장점들이 존재해요.   
204    204                         가족이 머물기에 전반적으로 만족했던 숙소였어요.   
205    205   줄 서 있던 시민들과 사진 촬영을 하던 백종원 씨는 대기줄에서 낯익은 얼굴을 발견했다.   
206    206                             정말 내가 지내본 숙소 중에 최악입니다.   
207    207                            큰 창과 아기자기한 방이 인상적이었습니다.   
208    208                        뻔한듯한 내용이지만 대사 한마디가 마음을 울린다.   
209    209                    충돌로 가벼운 상처를 입은 강씨 외에는 부상자는 없었다.   
210    210                           호스트의 아버지, 어머니는 정말 친절하셨다.   
211    211  일본 정부가 지난 27일 폭발한 규슈 지역의 신모에다케 화산 인근 주민들에게 피난 ...   
212    212                          장선우 감독과 이정현의 처음이자 마지막의 역작   
213    213                              손소독 후 앵무새들을 천천히 살펴봤다.   
214    214               

In [8]:
from transformers import AutoTokenizer
model_name = 'klue/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
from tqdm import tqdm
import numpy as np

def convert_examples_to_features(sent_list1,sent_list2, max_seq_len, tokenizer):
    input_ids, attention_masks, token_type_ids = [], [], []

    for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
        encoding_result = tokenizer.encode_plus(sent1, sent2,
                                                max_length=max_seq_len, pad_to_max_length=True)
        input_ids.append(encoding_result['input_ids'])
        attention_masks.append(encoding_result['attention_mask'])
        token_type_ids.append(encoding_result['token_type_ids'])
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    return (input_ids, attention_masks, token_type_ids)


In [10]:
X_train = convert_examples_to_features(train['premise'], train['hypothesis'], 
                                       max_seq_len=max_seq_len, 
                                       tokenizer=tokenizer)

100%|██████████| 50/50 [00:00<00:00, 2631.41it/s]


In [11]:
X_valid = convert_examples_to_features(valid['premise'], valid['hypothesis'], 
                                       max_seq_len=max_seq_len, 
                                       tokenizer=tokenizer)

100%|██████████| 50/50 [00:00<00:00, 2757.05it/s]


In [12]:
X_test = convert_examples_to_features(test['premise'], test['hypothesis'], 
                                       max_seq_len=max_seq_len, 
                                       tokenizer=tokenizer)

100%|██████████| 1666/1666 [00:00<00:00, 5074.32it/s]


In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(train['label'])
y_valid = le.transform(valid['label'])

In [14]:
label_idx = dict(zip(list(le.classes_), le.transform(list(le.classes_))))
label_idx

{'contradiction': 0, 'entailment': 1, 'neutral': 2}

In [15]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import TruncatedNormal
from transformers import TFAutoModel

class TFBertForSequenceClassification(Model):
    def __init__(self, model_name):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFAutoModel.from_pretrained(model_name, 
                                                num_labels=3, 
                                                from_pt=True)
        self.classifier = Dense(3,
                                kernel_initializer=TruncatedNormal(0.02),
                                activation='softmax')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids=inputs
        outputs = self.bert(input_ids=input_ids, 
                            attention_mask=attention_mask, 
                            token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [16]:
import tensorflow as tf

model = TFBertForSequenceClassification(model_name)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics='accuracy')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream 

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001,
    patience=2)

model.fit(
    X_train, y_train, epochs=2, batch_size=32, validation_data=(X_valid, y_valid),
    callbacks = [early_stopping]
)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2/2 [==============================] - 18s 3s/step - loss: 1.1214 - accuracy: 0.3800 - val_loss: 1.2506 - val_accuracy: 0.3600
Epoch 2/2
